<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/TF_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создаем простой граф


In [0]:
import tensorflow as tf

x = tf.Variable (3, name="x")
y = tf.Variable (4, name="y")
f = x*x*y + y + 2


**Добавляем сессию**

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


**Упращенная сессия (с автозакрытием)**

In [3]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
  print(result)

42


**Глобальная инициализация**

In [4]:
import tensorflow as tf

x = tf.Variable(3, "x")
y = tf.Variable(2, "y")
z = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
  init.run()
  result = f.eval()
  print(result)

42


**Новый отдельный экземпляр графа **

In [5]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)
  y2 = tf.Variable(6)
  Z2 = x2*y2
  
  init = tf.global_variables_initializer()
  sess = tf.InteractiveSession()
  init.run()
  result2 = Z2.eval()
  print(result2)
  sess.close()
  

12


**Оценка в одном прогоне**

In [6]:
import tensorflow as tf
w = tf.constant(3)
x = w + 2
y = x + 5 
z = x * 3

init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)
 
  sess.close()


10
15


**Линейная регресия упрощенная**

In [3]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()   
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape

#print(housing.feature_names)
#print(housing.DESCR)
#print(housing.target.shape)

# np.ones создает новый array заполненный 1ми, np.c_ создает новый масив обьединяя 2
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant (housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant (housing.target.reshape(-1,1), dtype=tf.float32, name="y")

XT = tf.transpose(X)
theta = tf.matmul (tf.matmul (tf.matrix_inverse(tf.matmul(XT,X)), XT) , y)

with tf.Session() as sess:
  theta_value = theta.eval ()
  print(theta_value)

[[-3.7112991e+01]
 [ 4.3611991e-01]
 [ 9.4082914e-03]
 [-1.0654381e-01]
 [ 6.4201808e-01]
 [-4.0360574e-06]
 [-3.7822633e-03]
 [-4.2303962e-01]
 [-4.3648642e-01]]


**Вычисление градиента в ручную и автоматически**

In [0]:




import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

m, n = housing.data.shape

n_epochs = 100000
learning_rate = 0.001

# нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

# np.c_ сращивание матриц, np.ones создание матрицы заполненой нулями
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# tf.random_uniform генерировать случайный тензер с заданой формой и диапазоном
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") 
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y

# tf.reduce_mean вычисляет среднее значение
mse = tf.reduce_mean(tf.square(error), name="mse")

#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients (mse, [theta])[0]

# создает узел который будет присваивать переменной новое значение 
training_op = tf.assign(theta, theta - learning_rate * gradients) 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
           print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

 **Вывод матрицы/тензора **

In [0]:
import tensorflow as tf
g = tf.random_uniform([10 + 1, 1], 0.0, 10.0, seed=42)
init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op) 
    print (sess.run(g))#распечатать тензор
    print(g.get_shape) #получить размер массива статич

       




**Использование оптимизатора**

In [71]:
import tensorflow as tf
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()
m, n = housing.data.shape

learn_rate = 0.01
epoch = 1000

Scaler = StandardScale()
Scaled_housing_data = Scaler.fit_transform(housing.data)

Scaled_housing_data_plus_bias = np.c_(tf.ones((m,1)), Scaled_housing_data)

X = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="x")
y = tf.constant(Scaled_housing_data_plus_bias, dtype=tf.float32, name="Y")

theta = tf.Variable((tf.random_uniform[n+1, 1], -1, 1, seed=32), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")
erorr = y_pred - Y
mse = tf.reduce_mean(tf.square(erorr), name="mse")



[4.526 3.585 3.521 ... 0.923 0.847 0.894]
[[4.526]
 [3.585]
 [3.521]
 ...
 [0.923]
 [0.847]
 [0.894]]
